<a href="https://colab.research.google.com/github/mattcturek/ML-For-Coders/blob/main/ML_for_Coders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 1

### Single Nuron Neural Network as a "Hello World!" application.

Verify the TensorFlow Version

In [ ]:
import tensorflow as tf;
print(tf.__version__)

Building a Neural Netowrk using TensorFlow and Keras APIs

In [ ]:
from keras.src.engine.training import optimizer
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))

The same code with it showing the weights and loss.

In [ ]:
from keras.src.utils.data_utils import SequenceEnqueuer
from keras.src.engine.training import optimizer

import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))

After running this code, it will you what it thinks the X is, and then what it thinks the relationship of Y to X.

## Chapter 2
### Computer Vision

In this tutorial, we will make a neural network that will look at an image of a piece of clothing, then categorize it and try to guess the correct category.

#### Neurons for Vision

In Chapter 1, you saw a very simple secenario where a machine was given a set of X and Y values, and it learned that the relationship between was Y=2X-1. This was done using a very simple neural network with one layer and one neuron.  

In this example we are uisng Fashion-MNIST dataset that comes with Keras. It is a set of images of clothing to train the model on. Each of our images is a set of 784 values (28 x 28) between 0 and 255. They can be our X.

We know that we have 10 different types of images in our dataset, so let's consider them to be our Y. Now we want to learn what the function looks like where Y is a function of X.

Below is the code for the model including the dataset.

In [ ]:
import tensorflow as tf
data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=25)
#model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)
print(classifications[9])
print(test_labels[9])

Here we can pick and choose which image in the test-images array to run through the neural network by using the

```
classifications = model.predict(test_images)  - loads the test images into the model.predict function
print(classifications['array-number']) - prints the softmax 10 neuron weights
print(test_label['matching-array-nunber']) - actual classification number
```



code.

The number in both array values must be the same to choose the same test image vs the same fashion category (0-9).

In [ ]:
classifications = model.predict(test_images)
print(classifications[5])
print(test_labels[5])

Play with the values of the classifications array and the test_labels array to discover different chances the neural network had a chance to guess the clothing type. You can see the weights of all 10 final neurons and then the actual type number (0-9).

You can see how accurate it is based off the number of times you train the network in the section above.

#### Stopping Trianing Automatically

In each of the cases so far, we've hardcoded the number of epochs we're training for. White that works, we might want to train until we reach the desired accuracy instead of constactly trying differentt numbers of epochs and training and retraining until we get to our desired value. So, for example, if we want to train until the model is at 95% accuracy on the training set, without knowing how many epochs that will take, how could we do that?

The easiest approach is to use a <i>callback</i> on the training. Let's take a look at the updated code that uses callbacks:

In [ ]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=50, callbacks=[callbacks])

First, we created a new class called 'myCallBack'. This takes a <b>tf.keras.callbacks.Callback</b> as a parameter. Ini t, we define the on_epoch_end function, which will give us details about hte logs for this epoch. In these logs is an accuracy value, so all we have to do is see if it is greater than 0.95 (or 95%); if it is, we can stop training by saying <code>self.model.stop_training = True</code>

Now check out the <code>model.fit</code> statement. You'll see that I've updated it to trian for 50 epochs, and then added a callbacks parameter. To this, I pass the <code>callbacks</code> object.

When training, at the end of every epoch, the callback function will be called. So at the end of each epoch you'll check, and after about 34 epochs you'll see that your training will end, becuase the training has hit 95% accuracy (your number may be slightly different becaues of the initial random initialization, but it will likely be quite close to 34)

# Chapter 3
### Dectecting Features in Images

### Convolutions

A convolution is simply a filter of weights that are used to multiply a pixel with its neighbors to get a new value for hte pixel.

In our example we are going to use the Fashion MNIST Data Set, yet we are going to create a filter that is 3x3 to cover each pixel in the image. We then multiply the pixel color (greyscale) by the filter numbers, then summed up to change the number representing that pixel. Then the filter is moved to the next pixel and the story repeats.

### Pooling

Pooling is the process of eliminating piels in your image while maintaining teh semantics of the content within the image. It's best explained visually.

Here is an example of <i>max</i> pooling.

Take a 4x4 image containing 16 pixels. Each group of 2x2 pixels in the upper left corner, upper right corner, lower left corner, and lower right corner. Each is 2x2. The highest number of each 2x2 array from the original image is processed and then sent to a 2x2 (4 pixel) array.

### Implementing Convultional Neural Networks

Here is the code from the original neural network that processed the same data. Now we are going to add convolutional layers and pooling layers to the neural network and check performance

<code>
import tensorflow as tf<br>
data = tf.keras.datasets.fashion_mnist<br><br>
(training _images, training_labels), (test_images, test_labels) = data.load_data()<br><br>
training_images = training_imagse / 255.0<br>
test_images = test_images / 255.0<br><br>
model = tf.keras.models.Sequential([<br>
tf.keras.layers.Flatten(input_shape=(28,28)),<br>
tf.keras.layers.Dense(128, activation=tf.nn.relu),<br>
tf.keras.layers.Densse(10, activation=tf.nn.softmax)<br>
])<br><br>
model.compile(optimizer='adam', loss='sparese_categorical_crossentropy', metrics=['accuracy'])<br><br>
model.fit(training_images, training_labes, epochs=5)
</code>
<br>
<br>
To implement a convolutional layer, you'll use the <code>tf.keras.layers.Conv2D</code> type.

For example, here's a convolutional layer used as the input layer to a neural network:
<br>
<code>
tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
</code>

In this case we want the layer to learn 64 convolutions. It will randomly initialize these, and over time will learn the filter values that work best wo match the input values to their labels. The (3, 3) indicates the size of the filter.

Here's how to use a pooling layer in the neural network. YOu'll typically do this immediately after the convolutional layer:
<code>tf.keras.layers.MaxPooling2D(2, 2)</code><br>

In this example we split the image into 2x2 pools and picked the maximum value in each. This operation could have been parameterized to define the pool size. those are the parameters that you can see here - the (2 x 2) indicates that our pools are 2 x 2.

### Code for Convolutional Neural Network

In [ ]:
import tensorflow as tf
data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

### Building a Convolutional Neural Network to Distinguish Between Horses and Humans

In this section we'll explore a more complex scenario than the Fashion MNIST classifier. We'll extend what we've learned about convolutions and convolutional neural networks to try to classify the contents of images where the location of a feature issnt' always in the sae place. I've created the Horses aor Humans dataset for this purpsose

### The Horses Or Humans Dataset

The dataset for this section (https://oreil.ly/E5kbc) contains over a thousand 300 x 300 pixel images, approximately half each of horses and humans, rendered in different poses. This link takes you to download the Learning Set of images.

### The Keras ImageDataGenerator

The Fashion MNIST dataset that you've been using up to this point comes with labels. Every timage file has an associated file with the label details. Many image-based datasets do not have this, and Horses or Humans is no exception.

Instea of labels, the images are sorted into subdirectories of each type. With Keras and Tensorflow, a tool called the <code>ImageDataGenerator</code> can use this structure to <i>automatically</i> assign labels to images.

Below is the code to get the training data and extract it into the appropriately named subdirectories.

This works if you don't have the dataset to upload to the venv.

In [ ]:
import urllib.request
import zipfile

url = 'horse-or-human\training'

file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

If you are not using this code to donwload and automatically unzip the file, then you will need to add this code to create the 'training_dir' folder:

<code>training_dir = 'horse-or-human/training/'</code>

To use the <code>ImageDataGenerator</code> we now simply use the follwoing code:

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the training_dir
training_dir = 'horse-or-human/training'

# All images will be rescaled by 1/255ths
train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size = (300, 300),
    class_mode = 'binary'
)

Found 1027 images belonging to 3 classes.


We first create an instance of the <code>ImageDataGenerator</code> called <code>train_datagen</code>. We then specify that this will generate images for the training process by flowing them from a directory. The directory is <i>training_dir</i>, as specififed earlier. We also indicate some hyperparameters about the data, such as the target size - in this case the images are 300 x 300, and the class modei s <code>binary</code>. The mode is usually <code>binary</code> if there are just two types of images (as in this case) or <code>categorical</code> if there are more than two.

### Convolutional Neural Network Architecture

There are major differences between this dataset and the Fashion MNIST one that you have to take into account when designing an achitecture for classifying the images. First, the images are musch larger - 300 x 300 pixels - so mroe layers  may be needed.

Second, the images are full color, not greyscale, so each image will have 3 channels instead of just 1.

Third, there are only two image types, so we have a binary classifier that can be implemented using just a single output neuron, where it approaches 0 for one class and 1 for another. Keep these considerations in mind when exploring this architecture:

In [ ]:
import tensorflow as tf
import keras


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

Compile the model with Binary Crossentropy loss factor, Root Mean Square Propagation as the optimizer, checking the 'accuracy' metric.

In [15]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

Then we will use a <b>fit_generator</b> and pass it the <code>training_generator</code> we created earlier:

In [16]:
history = model.fit_generator(train_generator, epochs = 15)

<ipython-input-16-373f6d72c78f>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs = 15)


Epoch 1/15
33/33 [==============================] - 104s 3s/step - loss: -79296016.0000 - accuracy: 0.4869
Epoch 2/15
33/33 [==============================] - 98s 3s/step - loss: -6748932096.0000 - accuracy: 0.4869
Epoch 3/15
33/33 [==============================] - 95s 3s/step - loss: -80535281664.0000 - accuracy: 0.4869
Epoch 4/15
33/33 [==============================] - 95s 3s/step - loss: -442142916608.0000 - accuracy: 0.4869
Epoch 5/15
33/33 [==============================] - 96s 3s/step - loss: -1629855154176.0000 - accuracy: 0.4869
Epoch 6/15
33/33 [==============================] - 97s 3s/step - loss: -4800283410432.0000 - accuracy: 0.4869
Epoch 7/15
33/33 [==============================] - 95s 3s/step - loss: -11837572644864.0000 - accuracy: 0.4869
Epoch 8/15
33/33 [==============================] - 96s 3s/step - loss: -25811260276736.0000 - accuracy: 0.4869
Epoch 9/15
33/33 [==============================] - 95s 3s/step - loss: -51870835933184.0000 - accuracy: 0.4869
Epoch 10

Run all the code in order but the beginning block where you download the images in a zip file. Then you can train the model, or you can copy and paste all the code into one code block to run.

### Adding Validation to the Horses or Humans Dataset

To add validation, you'll need a validation dataset that's separate from the training one. In some cases you'll get a master dataset that you have to split yourself, but in the case of Horses or Humans, there's a separate validation set that you can download.

# Appendix A

### Code Snippets

Train a Model:<br>
<code>model.fit(training_data, training_answers, epochs)</code>

Test Model on New Data:<br>
<code>model.evaluate(test_images, test_labels)</code>

Viewing one item in the array at a time and viewing the model's result vs the actual result:<br>
<code>
classifications = model.predict(test_images)<br>
print(classifications[9])<br>
print(test_labels[9])
</code>

Clean Up to terminate the Kernal and free up the used memory

<code>[ ] import os, signal<br>
os.kill(os.getpid(), signal.SIGKILL)
</code>